In [2]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import max_pool, max_pool_x, graclus, global_mean_pool, GCNConv,  global_mean_pool, SAGEConv, voxel_grid
import torch.nn.functional as F
import torch_scatter

class EventConv_mean_min_max_var(MessagePassing):

    def __init__(self, in_channels, out_channels):
        super(EventConv_mean_min_max_var, self).__init__()
        self.mlp = Seq(
            Linear(out_channels, out_channels), 
            ReLU(), 
            Linear(out_channels, out_channels)
        )

    def forward(self, x, edge_index):
        sara = self.propagate(edge_index, x=x)
        return sara

    def aggregate(self, inputs, index):
        sums = torch_scatter.scatter_add(inputs, index, dim=0)
        maxs = torch_scatter.scatter_max(inputs, index, dim=0)[0]
        means = torch_scatter.scatter_mean(inputs, index, dim=0)
        var = torch.relu(
            torch_scatter.scatter_mean(
                inputs ** 2,
                index,
                dim=0) -
            means ** 2)

        aggrs = torch.hstack((sums, maxs, means, var))
        return self.mlp(aggrs)
    
    
class edge_gcn(torch.nn.Module):
    def __init__(self):
        super(edge_gcn, self).__init__()

        in_dim = 3
        hidden_dim = in_dim*4
        self.edge_conv1 = EventConv_mean_min_max_var(in_dim, hidden_dim)

        in_dim2 = in_dim + hidden_dim
        hidden_dim2 = in_dim2 * 4
        ##print(in_dim2, hidden_dim2)
        self.edge_conv1_2L = EventConv_mean_min_max_var(in_dim2, hidden_dim2)

        in_dim3 = hidden_dim2 + in_dim2 
        hidden_dim3 = in_dim3 * 4 
        ##print(in_dim3, hidden_dim3)

        self.edge_conv1_3L = EventConv_mean_min_max_var(in_dim3, hidden_dim3)
        self.conv0 = GCNConv(hidden_dim3 + in_dim3, 64)
        self.bn0 = torch.nn.BatchNorm1d(64)

        self.conv1 = GCNConv(64, 16)
        self.bn1 = torch.nn.BatchNorm1d(16)
        self.conv2 = GCNConv(16, 16)
        self.bn2 = torch.nn.BatchNorm1d(16)
        
        self.fc1 = torch.nn.Linear(16 * 64, 128)
        self.fc2 = torch.nn.Linear(128, 2)


    def forward(self, data):

        source_data = data.x.clone().detach()
        data.x = self.edge_conv1(source_data, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1 = data.x
        ##print('part1', part1.shape)
        source_data_2L = torch.cat((source_data, part1), dim=1)
        ##print('source_data_2L', source_data_2L.shape)
        data.x = self.edge_conv1_2L(source_data_2L, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1_2L = data.x
        ##print('part1_2L', part1_2L.shape)
        source_data_3L = torch.cat((source_data_2L, part1_2L), dim=1)
        ##print('source_data_3L', source_data_3L.shape)
        data.x = self.edge_conv1_3L(
            source_data_3L, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1_3L = data.x
        ##print('part1_3L', part1_3L.shape)
        source_data_4L = torch.cat((source_data_3L, part1_3L), dim=1)
        ##print('source_data_4L', source_data_4L.shape)
                
        
        data.x = torch.sigmoid(self.conv0(source_data_4L, data.edge_index))
        data.x = self.bn0(data.x)
        data.x = torch.sigmoid(self.conv1(data.x, data.edge_index))
        data.x = self.bn1(data.x)
        data.x = torch.sigmoid(self.conv2(data.x, data.edge_index))
        data.x = self.bn2(data.x)
        
        cluster = voxel_grid(data.pos, batch=data.batch, size=0.25)
        x,_ = max_pool_x(cluster, data.x, batch=data.batch, size=64)
        
        x = x.view(-1, self.fc1.weight.size(1))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return x  # data.x

In [3]:
model = edge_gcn().cuda()
model

edge_gcn(
  (edge_conv1): EventConv_mean_min_max_var()
  (edge_conv1_2L): EventConv_mean_min_max_var()
  (edge_conv1_3L): EventConv_mean_min_max_var()
  (conv0): GCNConv(375, 64)
  (bn0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): GCNConv(64, 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(16, 16)
  (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [4]:
from torch_geometric.transforms import BaseTransform
class EdgeDropout(BaseTransform):
    def __init__(self, p=0.5):
        assert p > 0 and p < 1
        self.p = p
    
    def __call__(self, data):

        edge_mask = torch.rand(data.edge_index.size(1), device=data.edge_index.device) >= self.p
        data.edge_index = data.edge_index[:, edge_mask]
        data.edge_attr = data.edge_attr[edge_mask, :]
        
        return data
    
        

In [42]:
tr = EdgeDropout(0.4)

In [1]:
from imports.TrainModel import TrainModel
#!rm -rf ../data/contact_extraction7/{train,test,val}/processed/
tm = TrainModel('/home/hussain/tactile/data/contact_extraction7/', model, features='coords', lr=0.001, batch=1, transform=tr, augment=True)
#tm.train()

NameError: name 'model' is not defined

In [44]:
tm.train()

training:   0%|          | 0/150 [00:00<?, ?epoch/s]

  0%|          | 0/386 [00:00<?, ?batch/s]

logging


  0%|          | 0/386 [00:00<?, ?batch/s]

logging


...

logging


  0%|          | 0/386 [00:00<?, ?batch/s]

logging


  0%|          | 0/386 [00:00<?, ?batch/s]

logging


In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
tm.test()

In [ ]:
import numpy as np

np.degrees(0.021825013953680895)

In [ ]:
157, 124